In [1]:
import pandas as pd
import numpy as np
import re

# SLA Preprocessing

Create a data structure (dictionary) to match origin and destination to number of working days in the SLA.

In [2]:
sla_df = pd.read_excel('SLA_matrix.xlsx',header=1,index_col=1)
sla_df.head()

,Unnamed: 0,Metro Manila,Luzon,Visayas,Mindanao
Metro Manila,Origin\n(Seller),3 working days,5 working days,7 working days,7 working days
Luzon,NaN,5 working days,5 working days,7 working days,7 working days
Visayas,NaN,7 working days,7 working days,7 working days,7 working days
Mindanao,NaN,7 working days,7 working days,7 working days,7 working days
NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
sla_df = sla_df.loc["Metro Manila":"Mindanao","Metro Manila":"Mindanao"]
sla_df.head()

,Metro Manila,Luzon,Visayas,Mindanao
Metro Manila,3 working days,5 working days,7 working days,7 working days
Luzon,5 working days,5 working days,7 working days,7 working days
Visayas,7 working days,7 working days,7 working days,7 working days
Mindanao,7 working days,7 working days,7 working days,7 working days


In [4]:
origins = ["Metro Manila","Luzon","Visayas","Mindanao"]
destinations = origins

sla = {}
for origin in origins:
    for destination in destinations:
        working_days_string = sla_df[origin][destination]
        
        for character in working_days_string.split():
            if character.isdigit(): 
                working_days = int(character)  
                
        sla[(origin,destination)] = working_days
        
print(sla)


{('Metro Manila', 'Metro Manila'): 3, ('Metro Manila', 'Luzon'): 5, ('Metro Manila', 'Visayas'): 7, ('Metro Manila', 'Mindanao'): 7, ('Luzon', 'Metro Manila'): 5, ('Luzon', 'Luzon'): 5, ('Luzon', 'Visayas'): 7, ('Luzon', 'Mindanao'): 7, ('Visayas', 'Metro Manila'): 7, ('Visayas', 'Luzon'): 7, ('Visayas', 'Visayas'): 7, ('Visayas', 'Mindanao'): 7, ('Mindanao', 'Metro Manila'): 7, ('Mindanao', 'Luzon'): 7, ('Mindanao', 'Visayas'): 7, ('Mindanao', 'Mindanao'): 7}


# Actual Data

## Preprocessing

* Convert dates from epoch GMT+8 time to dates
* Remove time which is ignored
* Extract origin and destination from addresses

In [5]:
df = pd.read_csv("delivery_orders_march.csv")
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [6]:
df_subset = df
entries = df_subset.shape[0]
df_subset.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [7]:
df_subset['pick'] = pd.to_datetime(df_subset['pick'],unit='s') + pd.Timedelta('08:00:00')
df_subset['1st_deliver_attempt'] = pd.to_datetime(df_subset['1st_deliver_attempt'],unit='s') + pd.Timedelta('08:00:00')
df_subset['2nd_deliver_attempt'] = pd.to_datetime(df_subset['2nd_deliver_attempt'],unit='s') + pd.Timedelta('08:00:00')
df_subset.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02 16:39:57,2020-03-05 13:09:18,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04 16:19:28,2020-03-06 10:53:56,2020-03-10 08:02:56,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04 15:20:34,2020-03-06 09:56:19,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05 22:36:56,2020-03-07 12:45:41,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04 18:38:25,2020-03-06 15:41:40,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [8]:
df_subset['pick'] = df_subset['pick'].dt.date
df_subset['1st_deliver_attempt'] = df_subset['1st_deliver_attempt'].dt.date
df_subset['2nd_deliver_attempt'] = df_subset['2nd_deliver_attempt'].dt.date
df_subset.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04,2020-03-06,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05,2020-03-07,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04,2020-03-06,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [9]:
#Buyer address is destination, seller address is origin
conditions = [
    (df_subset['buyeraddress'].str.contains('Metro Manila',flags=re.IGNORECASE)),
    (df_subset['buyeraddress'].str.contains('Luzon',flags=re.IGNORECASE)),
    (df_subset['buyeraddress'].str.contains('Visayas',flags=re.IGNORECASE)),
    (df_subset['buyeraddress'].str.contains('Mindanao',flags=re.IGNORECASE))
]
choices = ["Metro Manila","Luzon","Visayas","Mindanao"]
df_subset['destination'] = np.select(conditions, choices)

conditions = [
    (df_subset['selleraddress'].str.contains('Metro Manila',flags=re.IGNORECASE)),
    (df_subset['selleraddress'].str.contains('Luzon',flags=re.IGNORECASE)),
    (df_subset['selleraddress'].str.contains('Visayas',flags=re.IGNORECASE)),
    (df_subset['selleraddress'].str.contains('Mindanao',flags=re.IGNORECASE))
]
df_subset['origin'] = np.select(conditions, choices)

df_subset.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,destination,origin
0,2215676524,2020-03-02,2020-03-05,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,Metro Manila,Metro Manila
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",Metro Manila,Metro Manila
2,2220979489,2020-03-04,2020-03-06,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",Metro Manila,Metro Manila
3,2221066352,2020-03-05,2020-03-07,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",Metro Manila,Metro Manila
4,2222478803,2020-03-04,2020-03-06,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,Luzon,Metro Manila


## Algorithm

* Find number of actual working days taken for each delivery attempt
* Compare actual working days taken to that stipulated in SLA to determine if the order is late.

Create a column to indicate number of working days in SLA according to origin and destination.

In [10]:
df_subset['origin_destination'] = tuple(zip(df_subset['origin'],df_subset['destination']))
conditions = [df_subset['origin_destination']==key for key,values in sla.items()]
choices = [values for key,values in sla.items()]

df_subset['working_days'] = np.select(conditions,choices)

df_subset.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,destination,origin,origin_destination,working_days
0,2215676524,2020-03-02,2020-03-05,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,Metro Manila,Metro Manila,"(Metro Manila, Metro Manila)",3
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",Metro Manila,Metro Manila,"(Metro Manila, Metro Manila)",3
2,2220979489,2020-03-04,2020-03-06,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",Metro Manila,Metro Manila,"(Metro Manila, Metro Manila)",3
3,2221066352,2020-03-05,2020-03-07,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",Metro Manila,Metro Manila,"(Metro Manila, Metro Manila)",3
4,2222478803,2020-03-04,2020-03-06,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,Luzon,Metro Manila,"(Metro Manila, Luzon)",5


Check the number of days taken for the first attempted delivery and possible second attempted delivery. Note: Also need to account for non-working days (Sunday) and public holidays.

In [11]:
def get_time_taken(start_column,end_column):
    public_holidays = ['2020-03-25','2020-03-30','2020-03-31'] #Don't count 2020-03-08 because it is a Sunday
    
    start = df_subset[start_column]
    end = df_subset[end_column]
    
    end = np.where(pd.isna(end),start + pd.Timedelta('1 days'),end)
    time_taken = np.busday_count(start.tolist(),end.tolist(),weekmask='Mon Tue Wed Thu Fri Sat',holidays=public_holidays)
    #return time_taken - non_working_days_count
    return time_taken
    
df_subset['1st_time_taken'] = get_time_taken('pick','1st_deliver_attempt')
df_subset['2nd_time_taken'] = get_time_taken('1st_deliver_attempt','2nd_deliver_attempt')

df_subset[5:10]

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,destination,origin,origin_destination,working_days,1st_time_taken,2nd_time_taken
5,2222597288,2020-03-04,2020-03-07,NaT,C-846 park&shop opens Valeda/Sunrise) CompuWar...,33 sta.mesa 1929 hawthorn Metro Manila,Metro Manila,Metro Manila,"(Metro Manila, Metro Manila)",3,3,1
6,2222738456,2020-03-02,2020-03-05,2020-03-09,"Ka-insong stockholme Lhuilier, polanco. malen ...","salcedo Midlands 980 WALL, 16C paraiso Mall（Ph...",Metro Manila,Metro Manila,"(Metro Manila, Metro Manila)",3,3,3
7,2224695304,2020-03-02,2020-03-10,NaT,Reel zambo.ciyy puyat.cor road/infront #129A L...,Polar 75 mulawinan 1G-3 señora Text/Call 2P-01...,Metro Manila,Metro Manila,"(Metro Manila, Metro Manila)",3,7,1
8,2224704587,2020-03-04,2020-03-05,2020-03-09,"Lumber,Naprada POT kadir Bagyan blanga FIRENDS...",skies dept. Crystal apello Stockroom.Star (DAG...,Luzon,Metro Manila,"(Metro Manila, Luzon)",5,1,3
9,2225138267,2020-03-04,2020-03-10,NaT,"Hengtong Buhaynasapa, Lask Southville8A Colgat...",9x Comfort Greenheights Lifehomes AANI Metro M...,Visayas,Metro Manila,"(Metro Manila, Visayas)",7,5,1


*Checking if the order is late according to SLA.*

Conditions:
* First attempted delivery from the pickup is more than number of working days in SLA.
* Second attempted delivery from first attempted delivery is more than 3 working days.

In [12]:
first_delivery_late = (df_subset['1st_time_taken'] > df_subset['working_days'])
df_subset['2nd_working_days'] = pd.DataFrame(3*np.ones(entries))
second_delivery_late = (df_subset['2nd_time_taken'] > df_subset['2nd_working_days'])
late_condition = first_delivery_late | second_delivery_late

df_subset['is_late'] = np.where(late_condition, 1, 0)

df_subset.head()


,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,destination,origin,origin_destination,working_days,1st_time_taken,2nd_time_taken,2nd_working_days,is_late
0,2215676524,2020-03-02,2020-03-05,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,Metro Manila,Metro Manila,"(Metro Manila, Metro Manila)",3,3,1,3.0,0
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",Metro Manila,Metro Manila,"(Metro Manila, Metro Manila)",3,2,3,3.0,0
2,2220979489,2020-03-04,2020-03-06,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",Metro Manila,Metro Manila,"(Metro Manila, Metro Manila)",3,2,1,3.0,0
3,2221066352,2020-03-05,2020-03-07,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",Metro Manila,Metro Manila,"(Metro Manila, Metro Manila)",3,2,1,3.0,0
4,2222478803,2020-03-04,2020-03-06,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,Luzon,Metro Manila,"(Metro Manila, Luzon)",5,2,1,3.0,0


Output order ID.

In [13]:
output = df_subset[['orderid','is_late']]
output.to_csv('output.csv',index=False)